In [1]:
!pip install hyperopt

In [14]:
import pandas as pd
import numpy as np

import datetime
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/DW.Matrix/Matrix_Three/DW_Matrix_Road_Sign'

/content/drive/My Drive/Colab Notebooks/DW.Matrix/Matrix_Three/DW_Matrix_Road_Sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, Y_train = train['features'], train['labels']
X_test, Y_test = test['features'], test['labels']

In [0]:
if Y_train.ndim == 1:
  Y_train = to_categorical(Y_train)

if Y_test.ndim == 1:
  Y_test = to_categorical(Y_test)

  input_shape = X_train.shape[1:]
num_classes = Y_train.shape[1]

In [0]:
def train_model(modl, X_train, Y_train, params_fit = {}):
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)
  
  model.fit(
      X_train, 
      Y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, Y_train)),
      callbacks = [tensorboard_callback]
  )
  return model

def predict(model_trained, X_test, Y_test, scoring = accuracy_score):
  Y_pred_prob = model_trained.predict(X_test)
  Y_pred = np.argmax(Y_pred_prob, axis = 1)
  Y_test_norm = np.argmax(Y_test, axis = 1)
  return scoring(Y_test_norm, Y_pred)

In [0]:
def get_cnn_v7(input_shape, num_classes):
  return Sequential([
       Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),             
       Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),             
       MaxPool2D(),
       Dropout(0.3),

       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
       MaxPool2D(),
       Dropout(0.3),

       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
       MaxPool2D(),
       Dropout(0.3),

       Flatten(),
       
       Dense(1024, activation = 'relu'),
       Dropout(0.3),

       Dense(1024, activation = 'relu'),
       Dropout(0.3),

       Dense (num_classes, activation = 'softmax')
  ])

In [8]:
model = get_cnn_v7(input_shape, num_classes)
model_trained = train_model(model, X_train, Y_train)

predict(model_trained, X_test, Y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.8109 - accuracy: 0.2418 - val_loss: 1.3891 - val_accuracy: 0.5756
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.1133 - accuracy: 0.6398 - val_loss: 0.3232 - val_accuracy: 0.8980
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.4896 - accuracy: 0.8432 - val_loss: 0.1202 - val_accuracy: 0.9667
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3248 - accuracy: 0.8990 - val_loss: 0.1125 - val_accuracy: 0.9729
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2357 - accuracy: 0.9289 - val_loss: 0.0476 - val_accuracy: 0.9853


0.9551020408163265

In [10]:
model_trained.evaluate(X_test, Y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1618 - accuracy: 0.9551


0.9551020264625549

In [0]:
def get_model(params):
  return Sequential([
       Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),             
       Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),             
       MaxPool2D(),
       Dropout(params['dropout_cnn_block_one']),

       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
       Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
       MaxPool2D(),
       Dropout(params['dropout_cnn_block_two']),

       Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
       Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
       MaxPool2D(),
       Dropout(params['dropout_cnn_block_three']),

       Flatten(),
       
       Dense(1024, activation = 'relu'),
       Dropout(params['dropout_dense_block_one']),

       Dense(1024, activation = 'relu'),
       Dropout(params['dropout_dense_block_two']),

       Dense (num_classes, activation = 'softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train, 
      Y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
  )

  accuracy = model.evaluate(X_test, Y_test)[0]
  print(params, 'accuracy = {}'.format(accuracy))
  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [24]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0, 1),
    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0, 1)
}


best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 3.2006 - accuracy: 0.0000e+00

 13/138 [=>............................]
 - ETA: 0s - loss: 2.9338 - accuracy: 0.0000e+00

 26/138 [====>.........................]
 - ETA: 0s - loss: 3.1167 - accuracy: 0.0000e+00

 38/138 [=======>......................]
 - ETA: 0s - loss: 2.9049 - accuracy: 0.0000e+00

 51/138 [==========>...................]
 - ETA: 0s - loss: 3.0973 - accuracy: 0.0055    

 65/138 [=============>................]
 - ETA: 0s - loss: 2.7554 - accuracy: 0.1856


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train, 
      Y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
  )

  score = model.evaluate(X_test, Y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy = {}'.format(accuracy))
  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [29]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0, 1),
    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0, 1)
}


best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.9255720442040554, 'dropout_cnn_block_three': 0.03244643501711875, 'dropout_cnn_block_two': 0.6574323462839613, 'dropout_dense_block_one': 0.12635719153746383, 'dropout_dense_block_two': 0.08487241732886208}
accuracy = 0.3265306055545807
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.8136848277973484, 'dropout_cnn_block_three': 0.28935402471564164, 'dropout_cnn_block_two': 0.267225872048086, 'dropout_dense_block_one': 0.2877582672714748, 'dropout_dense_block_two': 0.4078312109901948}
accuracy = 0.6267573833465576
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.8404486206729366, 'dropout_cnn_block_three': 0.5919328822796666, 'dropout_cnn_block_two': 0.8429236330908294, 'dropout_dense_block_one': 0.24937593655365264, 'dropout_dense_block_two': 0.8974040298586679}
accuracy = 0.08820861577987671
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.30923912080671445, 'dropout_cnn_block_three': 0.524929116961008, 'dropout_cnn_block_two': 0.23612

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
  
  model.fit(
      X_train, 
      Y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
  )

  score = model.evaluate(X_test, Y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy = {}'.format(accuracy))
  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [31]:
space = {
    'batch_size' : hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}


best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.49796959303539434, 'dropout_cnn_block_three': 0.3226846201057338, 'dropout_cnn_block_two': 0.3058822092790635, 'dropout_dense_block_one': 0.6504990333050267, 'dropout_dense_block_two': 0.5598568920787985}
accuracy = 0.9038548469543457
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3486222910353553, 'dropout_cnn_block_three': 0.325387087632793, 'dropout_cnn_block_two': 0.3185345073421547, 'dropout_dense_block_one': 0.565404719496752, 'dropout_dense_block_two': 0.38648492392863876}
accuracy = 0.9614512324333191
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.40265817611191757, 'dropout_cnn_block_three': 0.3046648233096629, 'dropout_cnn_block_two': 0.3279012773887217, 'dropout_dense_block_one': 0.34186315131711686, 'dropout_dense_block_two': 0.4853944833424161}
accuracy = 0.9553288221359253
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.36049026700791725, 'dropout_cnn_block_three': 0.48655810627397084, 'dropout_cnn_block_two': 0.446283